In [ ]:
%matplotlib inline
import numpy as np
import util
import multiprocessing
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 100
import matplotlib.pyplot as plt
import yt
import logging
logging.getLogger('yt').setLevel(logging.WARNING)
from yt.utilities.logger import ytLogger as mylog
print(yt.__version__)

from yt_synchrotron_emissivity import *


In [ ]:
fname = '/home/ychen/d9/FLASH4/2015_production_runs/0529_L45_M10_b1_h1/MHD_Jet_hdf5_plt_cnt_0600'
ds = yt.load(fname, particle_filename=fname.replace('plt_cnt', 'part')+'_updated')

In [ ]:

proj_axis = 'x'
ptype = 'lobe'
nu = (150, 'MHz')

pars = add_synchrotron_dtau_emissivity(ds, ptype=ptype, nu=nu, proj_axis=proj_axis)
ad = ds.all_data()
tag = ad[ptype, 'particle_tag']
#filter = np.logical_and((tag % 2 == 0), (ad[ptype, 'particle_shok']==0))
#filter = ad[ptype, 'particle_shok']==0
filter = np.logical_and((ad[ptype, 'particle_shok']==0), (abs(ad[ptype, 'particle_posx']) < 3.08567758E21))
#filter = (tag % 1 == 0)
y = ad[ptype, 'particle_position_y'][filter]/3.08567758E21
z = ad[ptype, 'particle_position_z'][filter]/3.08567758E21
emis = np.log10(np.abs(ad[ptype, 'particle_sync_spec_%.1f%s' % nu][filter]))
#plt.hist(tag, bins=140)
#plt.scatter(ad['all', 'particle_tadd'], ad['all', 'particle_tag'], s=1,linewidth=0)
fig=plt.figure(figsize=(9.3,15),dpi=200)
ax=fig.add_subplot(111)
ext = np.array([-20, 20, -40, 40])
ax.set_xlim(ext[0],ext[1])
ax.set_ylim(ext[2],ext[3])
ax.annotate('%6.3f Myr' % (float(ds.current_time)/3.15569E13),\
            (1,0), xytext=(0.05, 0.96),  textcoords='axes fraction',\
            horizontalalignment='left', verticalalignment='center')
sc=ax.scatter(y,z,s=1,c=emis,linewidth=0,cmap='algae',vmin=-19,vmax=-18.3,alpha=0.8)
cb=plt.colorbar(sc)
cb.set_label(u'log emissivity')
plt.xlabel('y (kpc)')
plt.ylabel('z (kpc)')
plt.show()

In [ ]:
null = plt.hist(emis, bins=100, range=(-20,-18.3))

In [ ]:
ptype = 'lobe'
pol = 'i'
nu = (150, 'MHz')
proj_axis = 'x'
norm = yt.YTQuantity(*nu).in_units('GHz').value**0.5
field = ('deposit', ('nn_emissivity_%s_%s_%%.1f%%s' % (pol, ptype)) % nu)
center=(0.0,0.0,0.0)


pars = add_synchrotron_dtau_emissivity(ds, ptype=ptype, nu=nu, proj_axis=proj_axis)
p=yt.SlicePlot(ds, proj_axis, field, center=center, origin='center-domain',\
                              width=((40,'kpc'), (80,'kpc')),)

In [ ]:
p.set_zlim(field, 1E-25, 1E-21)

p.annotate_grids(alpha=0.2)
p.annotate_particles((1,'kpc'), ptype='jetp')
p.show()

In [ ]:
p.annotate_clear()
p.annotate_grids(alpha=0.05)
#cmap = plt.cm.arbre
#cmap.set_bad('k')
p.set_cmap(field, 'arbre')
p.set_zlim(field, 1E-30, 1E-21)
p.show()

In [ ]:
gamc = (ad[(ptype, 'particle_dens')]/ad[(ptype, 'particle_den1')]) / ad[(ptype, 'particle_dtau')]
print ad[(ptype, 'particle_den1')][ind]
ind_gamc_nan = np.where(np.isnan(gamc))
ind = np.where(np.isnan(ad[ptype, 'particle_sync_spec_%.1f%s' % nu]))

#print ind_gamc_nan, ind

#for tadd, tag in zip(ad[ptype, 'particle_tadd'][ind], ad[ptype, 'particle_tag'][ind]):
#    print repr(tadd.v), tag.v

In [ ]:

#def _particle_pressure(field, data):
#    required = data['gas', 'pressure']
#    mylog.debug("Mapping grid data to particles for %s in %s" % (field, data))
#    return data.ds.find_field_values_at_points(('gas', 'pressure'), data[ptype, 'particle_position'])
#logging.getLogger('yt').setLevel(logging.INFO)
#ds.add_field((ptype, 'particle_pressure'), function=_particle_pressure, particle_type=True, units='g/(cm*s**2)')

me = yt.utilities.physical_constants.mass_electron #9.109E-28
c  = yt.utilities.physical_constants.speed_of_light #2.998E10
e  = yt.utilities.physical_constants.elementary_charge #4.803E-10 esu

gamma_min = yt.YTQuantity(10, 'dimensionless')
#nu = 1.5E8 # 150MHz

def _synchrotron_emissivity(field, data):
    # To convert from FLASH "none" unit to cgs unit, times the B field from FLASH by sqrt(4*pi)
    ptype = 'io'
    B = np.sqrt(data[(ptype, 'particle_magx')]**2+data[(ptype, 'particle_magy')]**2+data[(ptype, 'particle_magz')]**2)\
        *np.sqrt(4.0*np.pi)
    B = data.apply_units(B, 'G')
    nuc = 3.0*data[(ptype, 'particle_gamc')]**2*e*B/(4.0*np.pi*me*c)
    nu = data.get_field_parameter("frequency", default=yt.YTQuantity(1.4, 'GHz'))
    #P = data[(ptype, 'particle_pressure')]
    #P = data[(ptype, 'particle_dens')]*yt.YTQuantity(1, 'dyne/cm**2')
    fit_const = 5.8
    norm = 0.5*B**1.5*e**3.5/(c**2.5*me**1.5*(4.*np.pi)**0.5)
    N0 = 3.0/me/c/c/(np.log(np.abs(data[(ptype, 'particle_gamc')]/gamma_min)))

    return N0*norm*fit_const*nu**(-0.5)*np.exp(-nu/nuc)
#logging.getLogger('yt').setLevel(logging.DEBUG)
nu = yt.YTQuantity(1.4, 'GHz') # 1.4GHz
ds.add_field(('io', 'particle_emissivity'), function=_synchrotron_emissivity, particle_type=True, 
             display_name="%s Emissivity" % nu, force_override=True)

ds.add_particle_filter('jetp')
#ds.add_field(('jetp', 'particle_emissivity'), function=_synchrotron_emissivity, particle_type=True,
              #units='erg/s/cm**3/Hz')
data = ds.all_data(field_parameters={"frequency": nu})

In [ ]:
data = ds.all_data(field_parameters={"frequency": nu})
B = np.sqrt(data[('io', 'particle_magx')]**2+data[('io', 'particle_magy')]**2+data[('io', 'particle_magz')]**2)*np.sqrt(4.0*np.pi)
B = data.apply_units(B, 'G')
null = plt.hist(np.log10(B), bins=40, range=(-7,-3))

In [ ]:
Bgrid = data['gas', 'magnetic_field_strength']
print Bgrid.units
null = plt.hist(np.log10(Bgrid), bins=40, range=(-7,-3))

In [ ]:
filter = data[('io', 'particle_gamc')] > 0.0
print len(data[('io', 'particle_gamc')][filter])
null = plt.hist(np.log10(data[('io', 'particle_gamc')][filter]), bins=40, range=(0,8))

In [ ]:
nuc = 3.0*(data[('io', 'particle_gamc')])**2*e*B/(4.0*np.pi*me*c)
print nuc.in_units('Hz')
null = plt.hist(np.log10(nuc), bins=40, range=(5,16))

In [ ]:
data['jetp', 'particle_emissivity'].shape

In [ ]:
typ = 'jetp'
field='particle_emissivity'
filter = np.logical_and((data[typ, field] > 0.0), (data[typ, 'particle_shok']==0))
#filter = (ad['all', 'particle_shok']==0)
print len(data['io', field])
print len(data[typ, field])

print len(data[typ, field][filter])
print data[typ, field].max()
null = plt.hist(np.log10(data[typ, field][filter]), bins=40, range=(-40,-22))

In [ ]:
ptype = 'jetp'
p = yt.ParticleProjectionPlot(ds, 'x', (ptype, 'particle_emissivity'), width=((40, 'kpc'),(80, 'kpc')), 
                              depth=(40, 'kpc'), aspect=1)
#p = yt.ParticlePlot(ds, (ptype, 'particle_position_y'), (ptype, 'particle_position_z'), (ptype, 'particle_emissivity'))
p.set_zlim((ptype, 'particle_emissivity'), 1E-34, 1E-22)
p.show()

In [ ]:
ds.known_filters['jetp'].filtered_type

In [ ]:
ds.add_deposited_particle_field(('all', 'particle_emissivity'), 'cic')
fields = 'all_cic_emissivity'

ds.field_info[('deposit', fields)].take_log = True
plot = yt.ProjectionPlot(ds, 'x', fields=fields)#, center=(0.0,0.0,6.17E22), width=(40,'kpc'))
#slice.set_log(fields, True)
#slice.set_cmap('jet ', 'gist_heat')
#slice.set_zlim(fields, 1E-16, 1E-5)
plot.zoom(12)
#plot.annotate_grids()
plot.set_origin('native')
plot.show()

In [ ]:
data_level_0 = ds.covering_grid(level=3, left_edge=ds.domain_left_edge,\
                                      dims=ds.domain_dimensions, fields=['jetp_cic_gamc'])

In [ ]:
dim=64
zoom=12
#obj = ds.arbitrary_grid(ds.domain_left_edge/zoom, ds.domain_right_edge/zoom, dims=[dim,dim,dim*2])
leftedge=np.array([-30,-30,-60])*3.08567758E21
rightedge=np.array([30,30,60])*3.08567758E21
obj = ds.arbitrary_grid(leftedge, rightedge, dims=[dim,dim,dim*2])
#print obj['jetp_cic_gamc'][dim/2,:,:].shape
#print obj.fcoords.shape
fig = plt.figure(figsize=(8,12),dpi=200)
#ext = np.array([-9.65E22*8, 9.65E22*8, -1.93E23*8, 1.93E23*8])/zoom/3.08567758E21
ext = np.array([-30, 30, -60, 60])
ims = plt.imshow(np.log10(np.nansum(obj['deposit', 'jetp_cic_emissivity'][:,:,:], axis=0)).transpose(), extent=ext, \
                 origin='lower', cmap='algae', vmin=-20, vmax=-9)#, interpolation='spline16')

cb=plt.colorbar(ims)
cb.set_label(u'log (emissivity) (1.4GHz, arbitrary unit)')
#plt.title('150MHz')
plt.xlabel('y (kpc)')
plt.ylabel('z (kpc)')

In [ ]:
obj['deposit', 'jetp_cic_emissivity'].min()

In [ ]:
d = None
#dim=64
#zoom=16
#def _gamc_interp(field, data):
#    #data._debug()
#    #dim=128
#    global d
#    d = data
#    #obj = data.ds.arbitrary_grid(data.LeftEdge, data.RightEdge, dims=data.ActiveDimensions)
#    obj = ds.arbitrary_grid(ds.domain_left_edge/zoom, ds.domain_right_edge/zoom, dims=[dim,dim,dim*2])
#    x0, y0, z0 = ds.domain_left_edge/zoom
#    x1, y1, z1 = ds.domain_left_edge/zoom
#    boundaries = [x0,x1,y0,y1,z0,z1]
#    interp = yt.utilities.linear_interpolators.TrilinearFieldInterpolator(obj['deposit', 'jetp_cic_gamc'], \
#                                                                          boundaries, ['x','y','z'], True)
#    interpolated_data = interp(data)

#    return interpolated_data

def _gamc_interp(field, data):
    dim=64
    obj = data.ds.arbitrary_grid([-9.65E22, -9.65E22, -1.93E23], [9.65E22, 9.65E22, 1.93E23], dims=[dim, dim, dim*2])
    x0 = obj.fcoords.reshape([dim,dim,dim*2,3])[:,:,:,0].min()
    x1 = obj.fcoords.reshape([dim,dim,dim*2,3])[:,:,:,0].max()
    y0 = obj.fcoords.reshape([dim,dim,dim*2,3])[:,:,:,1].min()
    y1 = obj.fcoords.reshape([dim,dim,dim*2,3])[:,:,:,1].max()
    z0 = obj.fcoords.reshape([dim,dim,dim*2,3])[:,:,:,2].min()
    z1 = obj.fcoords.reshape([dim,dim,dim*2,3])[:,:,:,2].max()
    boundaries = [x0,x1,y0,y1,z0,z1]
    interp = yt.utilities.linear_interpolators.TrilinearFieldInterpolator(obj['deposit', 'jetp_cic_emissivity'], \
                                                                          boundaries, ['x','y','z'], True)
    interpolated_data = interp(data)

    return interpolated_data

ds.add_field(('deposit', 'emissivity_interp'), function=_gamc_interp, take_log=True)

In [ ]:
#zoom=16
#dim=128
plot = yt.SlicePlot(ds, 'x', fields='emissivity_interp')
#plot.set_zlim('gamc_interp', 1E2, 1E5)
plot.zoom(12)
#slice.annotate_grids()
plot.set_origin('native')
plot.show()

In [ ]:
plot.set_zlim('emissivity_interp', 1E5, 1E20)
plot.show()

In [ ]:

finfo = ds.field_info["deposit", "gamc_interp"]

gs = ds.index.select_grids(ds.index.max_level)
print gs.shape
#print gs
for g in gs:
    if g.NumberOfParticles > 0:
        g0 = g
        break
print g0, g0.NumberOfParticles

#g0[("deposit", "gamc_interp")]
from yt.data_objects.grid_patch import AMRGridPatch
tr = super(AMRGridPatch, g0).__getitem__(("deposit", "gamc_interp"))
print tr.shape, g0.ActiveDimensions
#print tr
